# Práctica 2: Aprendizaje y selección de modelos de clasificación*

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

### Alumnos:

* Pablo Moreira Garcia
* Ruben Martinez Sotoca

Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo

En esta práctica estudiaremos los modelos más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los clasificadores resultantes. Además, veremos métodos de selección de modelos orientados a obtener una configuración óptima de hiperparámetros.

# 1. Preliminares

Antes de comenzar cargamos las librerías para que estén disponibles posteriormente:

In [ ]:
# Third party
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import make_scorer

# Local application
import utilidades_practica_2_ordinaria as utils

Además, fijamos una semilla para que los experimentos sean reproducibles:

In [ ]:
random_state = 27912

# 2. Carga de datos

Vamos a utilizar, como en la practica 1, los conjuntos de datos `Breast Cancer Wisconsin` y `Pima Indians Diabetes`

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index_col = "id"
target = "diagnosis"

data_Cancer = utils.load_data(filepath, index_col, target)

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index_col = None
target = "Outcome"

data_Diabetes = utils.load_data(filepath, index_col, target)

Comprobando que se ha cargado correctamente:

In [ ]:
data_Cancer.sample(5, random_state=random_state)

In [ ]:
data_Diabetes.sample(5, random_state=random_state)

A su vez, lo dividimos en variables predictoras (`X`) y variable clase (`y`):

In [ ]:
target = "diagnosis"

(X_Cancer, y_Cancer) = utils.divide_dataset(data_Cancer, target)

In [ ]:
target = "Outcome"

(X_Diabetes, y_Diabetes) = utils.divide_dataset(data_Diabetes, target)

Vamos a comprobar que se ha separado correctamente. Comenzamos con las variables predictoras:

In [ ]:
X_Cancer.sample(5, random_state=random_state)

In [ ]:
X_Diabetes.sample(5, random_state=random_state)

Y continuamos con la variable clase:

In [ ]:
y_Cancer.sample(5, random_state=random_state)

In [ ]:
y_Diabetes.sample(5, random_state=random_state)

Por último, dividimos el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado:

In [ ]:
stratify = y_Cancer
train_size = 0.7

(X_train_Cancer, X_test_Cancer, y_train_Cancer, y_test_Cancer) = train_test_split(X_Cancer, y_Cancer,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

In [ ]:
stratify = y_Diabetes
train_size = 0.7

(X_train_Diabetes, X_test_Diabetes, y_train_Diabetes, y_test_Diabetes) = train_test_split(X_Diabetes, y_Diabetes,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Y nos aseguramos que se ha realizado adecuadamente. Comenzamos con el conjunto de datos de entrenamiento:

In [ ]:
X_train_Cancer.sample(5, random_state=random_state)

In [ ]:
y_train_Cancer.sample(5, random_state=random_state)

In [ ]:
X_train_Diabetes.sample(5, random_state=random_state)

In [ ]:
y_train_Diabetes.sample(5, random_state=random_state)

Y finalizamos con el conjunto de datos de prueba:

In [ ]:
X_test_Cancer.sample(5, random_state=random_state)

In [ ]:
y_test_Cancer.sample(5, random_state=random_state)

In [ ]:
X_test_Diabetes.sample(5, random_state=random_state)

In [ ]:
y_test_Diabetes.sample(5, random_state=random_state)

# 3. Evaluación de modelos

Para la evaluación de los modelos vamos a utilizar validación cruzada, lo cual nos garantiza que los resultados no van a estar sesgados. Con esto lo que hacemos es dividir el conjunto de datos en `n_splits` particiones, de las cuales se elige una como test y las demas como train. Este proceso se repite `n_repeats` cada vezz con conjuntos de entrenamiento y de test distintos.

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

Antes de comenzar a utilizar los modelos, es necesario que almacenemos en el conjunto de entrenamiento de nuestras bases de datos en la variable que vamos a utilizar. De lo contrario, usaríamos todo el conjunto de datos y las pruebas sobre el conjunto test perderían su eficacia.

In [ ]:
X_Cancer = X_train_Cancer
y_Cancer = y_train_Cancer

Gracias a lo estudiado en la práctica anterior, vamos a generar los transformadores con los que crearemos los pipelines con los que trabajaremos más adelante.

In [ ]:
del_columns_wisconsin = utils.QuitarColumnasTransformer(['perimeter_mean', 'area_mean', 'compactness_mean', 'concave points_mean','radius_se','texture_se',
                                    'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
                                    'fractal_dimension_se', 'radius_worst', 'smoothness_worst', 'symmetry_worst', 'texture_worst', 'perimeter_worst', 
                                    'area_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'fractal_dimension_worst', 'Unnamed: 32'])

nan_anomalos_wisconsin=utils.AnomalosANanTransformer({'radius_mean':[6.981, 22.27], 'texture_mean':[10.38, 29.97],'smoothness_mean':[0.06251, 0.1326],
                                     'symmetry_mean':[0.1203, 0.2459], 'fractal_dimension_mean':[0.04996, 0.0795],'concavity_mean':[0, 0.2871] })

imp = SimpleImputer(missing_values=float('nan'), strategy='mean')

discretizer_wisconsin = KBinsDiscretizer(n_bins=5, strategy="uniform")

Basicamente como explicamos en la practica anterior, borramos las variables que no nos son necesarias como todas las `x_se` o `x_worst`, las que tienen alta correlacion como `perimeter_mean`, `area_mean`... y las que tienen un alto numero de valores anomalos o ruidosos como puede ser la variable `Unnamed: 32`

Los valores anomalos que tengan las distintas variables, que si que utilizamos, los cambiamos a nan gracias al segundo transformador. Y luego creamos un imputador que cambia los valores Nan por la media de esa variable.

Por ultimo utilizamos un discretizador de 5 conjuntos ya que como explicamos en la practica anterior al representar graficamente estas variables en una nube de putos y diferenciar por colores vamos a ver en algunas variables que hay segmentos bien diferenciados con solo un tipo de clase.

Y lo mismo para la base de datos de diabetes.

In [ ]:
X_Diabetes = X_train_Diabetes
y_Diabetes = y_train_Diabetes

In [ ]:
del_columns_pima = utils.QuitarColumnasTransformer(['SkinThickness', 'Insulin'])

nan_anomalos_pima=utils.AnomalosANanTransformer({'Glucose':[56,199], 'BloodPressure':[38,102], 'Pregnancies':[0,13],
                                      'BMI':[18.2,50], 'DiabetesPedigreeFunction':[0.078,1.182], 'Age':[21,64]})

imp = SimpleImputer(missing_values=float('nan'), strategy='mean')

discretizer_pima = KBinsDiscretizer(n_bins=5, strategy="kmeans")

Funcion utilizada para normalizar nuestra base de datos para el algoritmo de los vecinos más cercanos.

In [ ]:
def normalize(dataset):
    dataNorm=((dataset)/(dataset.max()))
    return dataNorm

# 4. Selección de modelos

En este apartado vamos a poder elegir los mejores hiperparametros gracias a un algoritmo de seleccion de modelos por fuerza bruta `GridSearch` al cual le pasamos un conjunto de hiperparametros (que nosotros seleccionamos) y este hace una evaluacion mediante validacion cruzada de todas las posibles combinaciones entre los hiperparametros para poder seleccionar los mejores vaalores.

Una vez obtenidos los mejores valores, evaluaremos los clasificadores para comprobar si la estimación es buena.

## Selección para Wisconsin

Con scoring estamos eligiendo las metricas que utilizaremos, en este caso vamos a utilizar `recall_score` y `roc_auc`.

In [ ]:
scoring = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label="M")}

Vamos a usar la función `utils.optimize_params` para encontrar los mejores hiperparámetros y asi optimizar los distintos algoritmos y al mismo tiempo mostrar los resultados de la validación cruzada para poder ver los distintos resultados que vamos a ir obteniendo.

### KNeighborsClassifier

Es un método que simplemente busca en las observaciones más cercanas a la que se está tratando de predecir y clasifica el punto de interés basado en la mayoría de datos que le rodean.

In [ ]:
estimator = KNeighborsClassifier()
pipeline = Pipeline(steps=[('del_columns_wisconsin',del_columns_wisconsin),('nan_anomalos_wisconsin', nan_anomalos_wisconsin),('imp', imp),
                           ('discretizer_wisconsin', discretizer_wisconsin),('knn', estimator)])

X_Cancer_Norm = normalize(X_Cancer)

weights = ["uniform", "distance"]

n_neighbors = [3,5,7,9,11,13,15,17,19]

#algorithm = ["auto", "ball_tree", "kd_tree", "brute"]

metric = ["euclidean", "manhattan"]
#NO UTILIZAMOS MINKOWSKI PORQUE LA MEDIDA MINKOWSKI ES UNA GENERALIZACION DE LA DISTANCIA EUCLIDEA Y MANHATTAN AÑADIENDO UN PARAMETRO P

#p =[1, 2, 3, 4, 5, 6, 7]

#leaf_size = [30, 40, 50, 60, 70, 80, 90, 100]

#n_jobs = default -1

k_neighbors_clf = utils.optimize_params(pipeline,X_Cancer_Norm, y_Cancer, cv,
                                        scoring=scoring,
                                        knn__n_neighbors=n_neighbors,
                                        knn__weights=weights,
                                        knn__metric=metric)

Algunos de los parametros que podriamos utilizar son:

* `weights`: La forma de calcular los pesos para las predicciones, existen 2 posibilidades.:
   - uniform : Peso uniforme, donde todos los puntos de cada grupo pesan lo mismo.

   - distance :El peso equivale a la inversa de la distancia entre los puntos, va a valer mas si los puntos están muy cerca que si están alejados.

   - [callable] : Tambien esta la posibilidad donde acepta un array de distancias.
* `n_neighbors`: Numero de grupos (neighbors) a utilizar, si no se especifica sería 5.
* `algorithm`: Algoritmos para procesar los vecinos más cercanos:

   - ‘ball_tree’ utilizará BallTree
    
   - ‘kd_tree’ utilizará KDTree

   - ‘brute’ utilizará busqueda con fuerza bruta.

   - ‘auto’ elegira de forma automatica el algoritmo más adecuado segun los valores que se le pasen al fit.
   
* `metric`: La medida de distancia que se va a utilizar, si no se especifica, se utiliza `Minkowski` donde utilizamos la variable `p`, si la `p` es 2 sería equivalente a utilizar `Euclidean`, y si fuera 1, sería equivalente a utilizar `Manhattan`.

* `leaf_size`: Tamaño de las hojas utilizado en los algooritmos `ball_tree` o `kd_tree`.

* `n_jobs`: El número de busquedas paralelas que va a realizar el ordenador, cuanto más alto mejor va a ser el resultado pero más costoso.

En este caso hemos utilizado unicamente `metric`, `n_neighbors` y `weighs`, donde los mejores hiperparametros han sido `euclidean`, `3` y `uniform` respectivamente, donde vemos que los resultados en cuanto a score no cambian, cuando se prueba con `euclidean` o `manhattan`, o con `uniform` o `distance`, pero lo que si que vemos que cambia son los tiempos.

### DecisionTreeClassifier

Los `árboles de decisión` son una técnica de aprendizaje automático supervisado muy utilizada. Como su nombre indica, esta técnica toma una serie de decisiones en forma de árbol. Los nodos finales (las hojas) nos dan la predicción que vamos buscando.

In [ ]:
# Should not modify the original model
estimator = DecisionTreeClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_wisconsin',del_columns_wisconsin),('nan_anomalos_wisconsin', nan_anomalos_wisconsin),('imp', imp),
                           ('discretizer_wisconsin', discretizer_wisconsin),('DecisionTree', estimator)])

criterion = ["gini", "entropy"]
max_depth = [4, 5, 6, 7, 8]

ccp_alpha = [0.0, 0.1, 0.2, 0.3]
#class_weight = ["balanced", None]
#max_features = [int, float, "auto", "sqrt", "log2", None]
#max_leaf_nodes = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, None]
#min_impurity_decrease = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
#min_samples_split = [10, 15, 20, 30, 40]
#min_samples_leaf = [5,6,7,8,9,10,11,12,13,14,15]
#min_weight_fraction_leaf = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]

splitter = ["best","random"]



decision_tree_clf = utils.optimize_params(pipeline,X_Cancer, y_Cancer, cv,
                                          scoring=scoring,
                                          DecisionTree__criterion=criterion,
                                          DecisionTree__max_depth=max_depth,
                                          DecisionTree__ccp_alpha=ccp_alpha,
                                          DecisionTree__splitter=splitter)

Algunos de los parametros que podriamos utilizar son:

* `criterion`: tipo de cadena, opcional (el valor predeterminado es "gini") Mide la calidad de la clasificación. Los estándares admitidos son "gini" para la impureza y "entropy" para la ganancia de información.

* `splitter`: {“best”, “random”}, default=”best”. Una estrategia utilizada para seleccionar categorías en nodos. Las estrategias admitidas son "best", seleccione la mejor categoría y "random" seleccione la mejor categoría aleatoria.

* `max_depth`: int, default=None. Profundidad máxima que puede alcanzar el árbol. Si no se especifica el arbol se expanderá hasta que todas las hojas sean de una única clase o mientras cada hoja contenga menos ejemplos que el parametro `min_samples_leaf` 

* `min_samples_split`: int or float, default=2. Número mínimo de observaciones que debe de tener un nodo para que pueda dividirse. Si es un valor decimal se interpreta como fracción del total de observaciones de entrenamiento.

* `min_samples_leaf`: int or float, default=1. número mínimo de muestras que debe haber en un nodo final (hoja). También se puede expresar en porcentaje.

* `min_weight_fraction_leaf`: float, opcional (el valor predeterminado es 0) La puntuación mínima ponderada requerida por la muestra de entrada de un nodo hoja.La puntuación mínima ponderada requerida por la muestra de entrada de un nodo hoja

* `max_features`: int, float, string o None opcional (el valor predeterminado es None) La cantidad de características que deben tenerse en cuenta al clasificar.
    1. Si es un int, las características de max_features deben ser consideradas en cada clasificación.
    
    2. Si es un flotante, entonces max_features es un porcentaje y la cantidad de características que se deben considerar durante la clasificación es int (max_features * n_features, donde n_features es la cantidad de características enviadas cuando se completa el entrenamiento).
    
    3. Si es automático, max_features = sqrt (n_features)
    
    4. Si es sqrt, max_features = sqrt (n_features)
    
    5. Si es log2, max_features = log2 (n_features)
    
    6. Si es None, max_features = n_features

    Nota: Cuando se encuentre al menos un punto de muestra clasificado, la búsqueda y clasificación se detendrá.

* `random_stateint`: Semilla para que los resultados sean reproducibles. Tiene que ser un valor entero.

* `max_leaf_nodes`: int, default=None Número máximo de nodos terminales.

* `min_impurity_decrease`: float, default=0.0 Un nodo se dividirá si esta division produce un decremento de la impureza igual o mayor a este valor.

* `min_impurity_split`:float, default=0 Limite para parar el crecimiento del arbol. Si un nodo esta por encima del limite se divide, si no, sería una hoja.

* `ccp_alphanon-negative`: float, default=0.0 Determina el grado de penalización por complejidad. Cuanto mayor es este valor, más agresivo el podado y menor el tamaño del árbol resultante.

En este caso hemos decidido utilizar `cpp_alpha`, `criterion` , `max_depth` y `splitter` y donde los hiperparametros utilizados son `0.0`, `entropy`, `8` y `random` respectivamente. Con cpp_alpha = 0.0 podemos ver que el podado va a ser minimo permitiendo que el arbol se expanda, con una profundidad maxima de 8. Y vemos que la categoria de los nodos se elige aleatoriamente.

### AdaBoostClassifier

A diferencia de muchos modelos ML que se enfocan en un solo modelo para completar predicciones de alta calidad, el algoritmo `Boosting` intenta mejorar las capacidades de predicción entrenando una serie de modelos débiles, cada uno de los cuales puede compensar las debilidades de sus predecesores.

`AdaBoost` es un algoritmo de `Boosting` específico (también conocido como `AdaBoost discreto`) desarrollado para problemas de clasificación. La debilidad está determinada por la tasa de error del estimador débil.

In [ ]:
estimator = AdaBoostClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_wisconsin',del_columns_wisconsin),('nan_anomalos_wisconsin', nan_anomalos_wisconsin),
                           ('imp', imp),('discretizer_wisconsin', discretizer_wisconsin),('AdaBoost', estimator)])

#algorithm = ["SAMME", "SAMME.R"]
base_estimator = [DecisionTreeClassifier(random_state=random_state)]
learning_rate = [0.1, 0.2,0.95, 1.0]
n_estimators = [100, 120, 150, 200]

criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]



adaboost_clf = utils.optimize_params(pipeline, X_Cancer, y_Cancer, cv,
                                     scoring=scoring,
                                     AdaBoost__base_estimator=base_estimator,
                                     AdaBoost__learning_rate=learning_rate,
                                     AdaBoost__n_estimators=n_estimators,
                                     AdaBoost__base_estimator__criterion=criterion,
                                     AdaBoost__base_estimator__max_depth=max_depth)

Algunos de los parametros que podriamos utilizar son:

* `base_estimator`: object, default=None El estimador base a partir del cual se construye el conjunto. Se requiere soporte para la ponderación de la muestra, así como atributos adecuados de classes_ y n_classes_ . Si es None, entonces el estimador base es DecisionTreeClassifier inicializado con max_depth = 1.

* `n_estimators`: int, default=50 El número máximo de estimadores en los que finaliza el boosting. En caso de un ajuste perfecto, el procedimiento de aprendizaje se detiene antes de tiempo.

* `learning_rate`: float, default=1. Peso aplicado a cada clasificador en cada iteración del boosting. Una tasa de aprendizaje más alta aumenta la contribución de cada clasificador. Existe una compensación entre los parámetros learning_rate y n_estimators.

* `algorithm`: {‘SAMME’, ‘SAMME.R’}, default=’SAMME.R’ Si es 'SAMME.R', utiliza el algoritmo de real boosting de SAMME.R. base_estimator debe soportar el cálculo de probabilidades de clase. Si es 'SAMME', utiliza el algoritmo de discrete boosting de SAMME. El algoritmo SAMME.R generalmente converge más rápido que SAMME, logrando un error de prueba menor con menos iteraciones de impulso.

* `random_state`: int, RandomState instance o None, default=None Controla la semilla aleatoria dada en cada base_estimator en cada iteración de boosting. Por lo tanto, solo se usa cuando base_estimator expone un random_state.

A parte de estos tambien existirian los parametros del estimador base.

Nosotros unicamente vamos a utilizar los hiperparametros `n_estimators`, `learning_rate` y `base_estimator` como `base_estimator` utilizamos arboles de decisión con `max_depth = 1` y `criterion = entropy`

### BaggingClassifier

`BagginClassifier` es un metaestimador de conjunto que ajusta los clasificadores base cada uno en subconjuntos aleatorios del conjunto de datos original y luego agrega sus predicciones individuales (ya sea por votación o promediando) para formar una predicción final. 
Un metaestimador de este tipo se puede utilizar típicamente como una forma de reducir la varianza de un estimador de caja negra (por ejemplo, un árbol de decisión), al introducir la aleatorización en su procedimiento de construcción y luego hacer un conjunto a partir de él.

In [ ]:
estimator = BaggingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_wisconsin',del_columns_wisconsin),('nan_anomalos_wisconsin', nan_anomalos_wisconsin),
                           ('imp', imp),('discretizer_wisconsin', discretizer_wisconsin),('Bagging', estimator)])

base_estimator = [DecisionTreeClassifier(random_state=random_state)]
n_estimators = [ 10, 50, 75, 100 ]
max_samples = [ 0.2, 1.0]
bootstrap = [ True, False ]


criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]


bagging_clf = utils.optimize_params(pipeline,X_Cancer, y_Cancer, cv,
                                    scoring=scoring,
                                    Bagging__n_estimators=n_estimators,
                                    Bagging__max_samples=max_samples,
                                    Bagging__bootstrap=bootstrap,
                                    Bagging__base_estimator=base_estimator,
                                    Bagging__base_estimator__criterion=criterion,
                                    Bagging__base_estimator__max_depth=max_depth)

Algunos de los parametros que podriamos utilizar son:

* `base_estimator`: object, default=None El estimador base para ajustarse a subconjuntos aleatorios del conjunto de datos. Si es None, entonces el estimador base es un DecisionTreeClassifier.

* `n_estimators`: int, default=10 El número de estimadores de base en el conjunto.

* `max_samples`: int or float, default=1.0 El número de muestras a extraer de X para entrenar a cada estimador base.

    Si es int entonces extrae `max_samples` muestras.

    Si es float entonces extrae `max_samples` * X.shape[0] muestras.

* `max_features`: int or float, default=1.0 El número de características a extraer de X para entrenar cada estimador base.

    Si es int entonces extrae `max_features` caracteristicas.

    Si es float entonces extrae`max_features` * X.shape[1] caracteristicas.

* `bootstrap`: bool, default=True. Las muestras se extraen con reemplazo. Si es Falso, se realiza un muestreo sin reemplazo.

* `bootstrap_features`: bool, default=False Si los rasgos se dibujan con el reemplazo.

* `oob_score`: bool, default=False Si utilizar muestras out-of-bag para estimar el error de generalización. Solo disponible si bootstrap=True.

* `warm_start`: bool, default=False Cuando es True, reutiliza la solución de la llamada anterior para ajustar y agregar más estimadores al conjunto, de lo contrario, simplemente ajusta un conjunto completamente nuevo.

* `n_jobs`: int, default=None El número de busquedas que se ejecutarán en paralelo tanto para ajustar como para predecir.

* `random_state`: int, RandomState instance o None, default=None Controla el remuestreo aleatorio del conjunto de datos original (en cuanto a muestras y características). Si el estimador base acepta un atributo de `random_state`, se genera una semilla diferente para cada instancia en el conjunto. Pasar un int para una salida reproducible a través de múltiples llamadas a funciones.

* `verbose`: int, default=0 Controla la verbosidad al ajustar y predecir.

A parte de estos tambien se podrían ajustar los parametros del estimador base.

Nosotros utilizamos los hiperparametros `bootstrap` para extraer las muestras con reemplazo, `max_samples` y `n_estimators`, a parte de algunos hiperparametros para nuesto estimador base que en este caso es un arbol de decisión.

### RandomForestClassifier

Es un conjunto (ensemble) de `árboles de decisión` combinados con `bagging`. Al usar `bagging`, lo que en realidad está pasando, es que distintos árboles ven distintas porciones de los datos. Ningún árbol ve todos los datos de entrenamiento. Esto hace que cada árbol se entrene con distintas muestras de datos para un mismo problema. De esta forma, al combinar sus resultados, unos errores se compensan con otros y tenemos una predicción que generaliza mejor.

In [ ]:
estimator = RandomForestClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_wisconsin',del_columns_wisconsin),('nan_anomalos_wisconsin', nan_anomalos_wisconsin),
                           ('imp', imp),('discretizer_wisconsin', discretizer_wisconsin),('RandForest', estimator)])

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
#max_depth = [5,6,7,8,9]
#class_weight = ["balanced", None]
n_estimators = [50, 100, 150, 200]
random_forest_clf = utils.optimize_params(pipeline,X_Cancer, y_Cancer, cv,
                                          scoring=scoring,
                                          RandForest__criterion=criterion,
                                          RandForest__n_estimators=n_estimators,
                                          RandForest__max_features=max_features)

Algunos de los parametros que podriamos utilizar son:

* `n_estimators`: int, default=100 número de árboles que va a tener el bosque aleatorio. Normalmente cuantos más mejor, pero a partir de cierto punto deja de mejorar y sólo hace que vaya más lento. Un buen valor por defecto puede ser el uso de 100 árboles.

* `criterion`: {“gini”, “entropy”}, default=”gini” Mide la calidad de la clasificación. Los estándares admitidos son "gini" para la impureza y "entropy" para la ganancia de información.

* `max_depth`: int, default=None La profundidad máxima del árbol.

* `min_samples_split`: int or float, default=2 número mínimo de muestras necesarias antes de dividir este nodo.

* `min_samples_leaf`: int or float, default=1 número mínimo de muestras que debe haber en un nodo final (hoja). También se puede expresar en porcentaje.

* `min_weight_fraction_leaf`: float, default=0.0 La puntuación mínima ponderada requerida por la muestra de entrada de un nodo hoja

* `max_features`:{“auto”, “sqrt”, “log2”}, int or float, default=”auto” La cantidad de características que deben tenerse en cuenta al clasificar.
    1. Si es un int, las características de max_features deben ser consideradas en cada clasificación.
    
    2. Si es un flotante, entonces max_features es un porcentaje y la cantidad de características que se deben considerar durante la clasificación es int (max_features * n_features, donde n_features es la cantidad de características enviadas cuando se completa el entrenamiento).
    
    3. Si es automático, max_features = sqrt (n_features)
    
    4. Si es sqrt, max_features = sqrt (n_features)
    
    5. Si es log2, max_features = log2 (n_features)
    
    6. Si es None, max_features = n_features

    Nota: Cuando se encuentre al menos un punto de muestra clasificado, la búsqueda y clasificación se detendrá.

* `max_leaf_nodes`: int, default=None número máximo de nodos finales

* `min_impurity_decrease`: float, default=0.0 Un nodo se dividirá si esta division produce un decremento de la impureza igual o mayor a este valor.

* `min_impurity_split`: float, default=None Limite para parar el crecimiento del arbol. Si un nodo esta por encima del limite se divide, si no, sería una hoja.

* `bootstrap`: bool, default=True Si se utilizan muestras de bootstrap al construir árboles. Si es False, se usa todo el conjunto de datos para construir cada árbol.

* `oob_score`: bool, default=False Si utilizar muestras out-of-bag para estimar el error de generalización. Solo disponible si bootstrap=True.

* `n_jobs`: int, default=None Número de cores que se pueden usar para entrenar los árboles. Cada árbol es independiente del resto, así que entrenar un bosque aleatorio es una tarea muy paralelizable. Por defecto sólo utiliza 1 core de la CPU. Para mejorar el rendimiento puedes usar tantos cores como estimes necesario. Si usas n_jobs = -1, estás indicando que quieres usar tantos cores como tenga tu máquina.

* `random_state`: int, RandomState instance or None, default=None Controla tanto la aleatoriedad del bootstrapping de las muestras utilizadas al construir árboles (si bootstrap = True) como el muestreo de las características a considerar cuando se busca la mejor división en cada nodo (si max_features <n_features).

* `verbose`: int, default=0 Controla la verbosidad al ajustar y predecir.

* `warm_start`: bool, default=False Cuando se establece en True, reutilice la solución de la llamada anterior para ajustar y agregar más estimadores al conjunto; de lo contrario, simplemente ajuste un bosque completamente nuevo.

* `class_weight`: {“balanced”, “balanced_subsample”}, diccionario o lista de diccionarios, default=None Pesos asociados con clases en el formato {class_label: weight}. Si no se da, se supone que todas las clases tienen un peso uno. Para problemas de múltiples salidas, se puede proporcionar una lista de dictados en el mismo orden que las columnas de y.

* `ccp_alpha`: non-negative float, default=0.0 Parámetro de complejidad utilizado para la poda Minimal Cost-Complexity. Se elegirá el subárbol con la mayor complejidad de costo siendo más pequeño que ccp_alpha. De forma predeterminada, no se realiza ninguna poda.
    
* `max_samples`: int or float, default=None Si bootstrap es True, El número de muestras a extraer de X para entrenar a cada estimador base.

    Si es None entonces extrae X.shape[0] muestras.

    Si es int entonces extrae `max_samples` muestras.

    Si es float entonces extrae `max_samples` * X.shape[0] muestras. Por lo tanto, max_samples debe estar en el intervalo (0, 1).

Para `random_forest` utilizamos los hiperparametros `criterion`, `max_features` y `n_estimators` y `gini`, `sqrt` y `150` respectivamente son los mejores hiperparametros seleccionados.

### GradientBoostingClassifier

Algoritmo basado en la combinación de modelos predictivos débiles (weak learners) normalmente árboles de decisión para crear un modelo predictivo fuerte. La generación de los árboles de decisión débiles se realiza de forma secuencial, creándose cada árbol de forma que corrija los errores del árbol anterior. Los aprendices suelen ser árboles "poco profundos", de apenas uno, dos o tres niveles de profundidad

In [ ]:
estimator = GradientBoostingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_wisconsin',del_columns_wisconsin),('nan_anomalos_wisconsin', nan_anomalos_wisconsin),
                           ('imp', imp),('discretizer_wisconsin', discretizer_wisconsin),('GradientBoosting', estimator)])

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
subsample = [0.2, 0.8, 1.0]
#ccp_alpha = [0.0, 0.1]
n_estimators = [50, 100, 200]

gradient_boosting_clf = utils.optimize_params(pipeline,X_Cancer, y_Cancer, cv,
                                              scoring=scoring,
                                              GradientBoosting__learning_rate=learning_rate,
                                              GradientBoosting__criterion=criterion,
                                              GradientBoosting__max_depth=max_depth,
                                              GradientBoosting__subsample=subsample,
                                              GradientBoosting__n_estimators = n_estimators)

Algunos de los parametros que podriamos utillizar son:

* `loss`: {‘deviance’, ‘exponential’}, default=’deviance’ La función `loss` a optimizar. "deviance" se refiere a la desviación (regresión logística) para la clasificación con resultados probabilísticos. "exponencial" gradientBoosting recupera el algoritmo AdaBoost.

* `learning_rate`: float, default=0.1 Reduce la contribución de cada árbol multiplicando su influencia original por este valor.

* `n_estimators`: int, default=100 El número de etapas de boosting a realizar. Gradient Boosting es bastante robusto ante el sobreajuste, por lo que un gran número generalmente da como resultado un mejor rendimiento.

* `subsample`: float, default=1.0 La proporción de submuestras utilizadas en el entrenamiento de cada árbol de decisión con respecto al total de muestras, y la selección de submuestras es aleatoria. El uso de un valor ligeramente inferior a 1 puede hacer que el modelo sea más robusto porque reduce la varianza.

* `criterion`: {‘friedman_mse’, ‘mse’, ‘mae’}, default=’friedman_mse’ La función para medir la calidad de una división. Los criterios admitidos son "friedman_mse" para el error cuadrático medio con puntuación de mejora de Friedman, "mse" para el error cuadrático medio y "mae" para el error absoluto medio.

* `min_samples_split`: int or float, default=2 Número mínimo de observaciones que debe de tener un nodo para que pueda dividirse. Si es un valor decimal se interpreta como fracción del total de observaciones de entrenamiento.

* `min_samples_leaf`: int or float, default=1 número mínimo de muestras que debe haber en un nodo final (hoja). También se puede expresar en porcentaje.

* `min_weight_fraction_leaf`: float, default=0.0 La fracción ponderada mínima de la suma total de pesos (de todas las muestras de entrada) que se requiere para estar en un nodo hoja. Las muestras tienen el mismo peso cuando no se proporciona sample_weight.

* `max_depth`: int, default=3 La profundidad máxima de los estimadores de regresión individuales.

* `min_impurity_decrease`: float, default=0.0 Un nodo se dividirá si esta división induce una disminución de la impureza mayor o igual a este valor.

* `min_impurity_split`: float, default=None Limite para parar el crecimiento del arbol. Si un nodo esta por encima del limite se divide, si no, sería una hoja.

* `random_state`: int, RandomState instance or None, default=None Controla la semilla aleatoria dada a cada Tree Estimator en cada iteración de boosting. Además, controla la permutación aleatoria de las características en cada división. 

* `max_features`: {‘auto’, ‘sqrt’, ‘log2’}, int or float, default=None La cantidad de características a considerar al buscar la mejor división:

    Si es int, entonces considere las características de max_features en cada división.

    Si es flotante, max_features es una fracción y las características int (max_features * n_features) se consideran en cada división.

    Si es "auto", entonces max_features = sqrt (n_features).

    Si es "sqrt", entonces max_features = sqrt (n_features).

    Si es "log2", entonces max_features = log2 (n_features).

    Si es None, entonces max_features = n_features.

    La elección de max_features <n_features conduce a una reducción de la varianza y un aumento del sesgo.

    Nota: la búsqueda de una división no se detiene hasta que se encuentra al menos una partición válida de las muestras de nodo, incluso si requiere inspeccionar de manera efectiva más de las características de max_features.

* `verbose`: int, default=0 Habilita la salida detallada. Si es 1, muestra el progreso y el rendimiento de vez en cuando (cuantos más árboles, menor es la frecuencia). Si es mayor que 1, imprime el progreso y el rendimiento de cada árbol.

* `max_leaf_nodes`: int, default=None Los árboles crecen con max_leaf_nodes numero de hojas de la mejor manera. Los mejores nodos se definen como una reducción relativa de la impureza. Si es None, entonces un número ilimitado de nodos hoja.

* `warm_start`: bool, default=False Cuando se establece en Verdadero, reutiliza la solución de la llamada anterior para ajustar y agregar más estimadores al conjunto; de lo contrario, simplemente borra la solución anterior. 

* `validation_fraction`: float, default=0.1 La proporción de datos de entrenamiento que se deben reservar como conjunto de validación para la detención anticipada. Debe estar entre 0 y 1.Solo se usa si n_iter_no_change se establece en un número entero.

* `n_iter_no_change`: int, default=None n_iter_no_change se utiliza para decidir si la parada anticipada se utilizará para finalizar el entrenamiento cuando la puntuación de validación no mejora. De forma predeterminada, está configurado en Ninguno para deshabilitar la parada anticipada. Si se establece en un número, dejará de lado el tamaño de validation_fraction de los datos de entrenamiento como validación y finalizará el entrenamiento cuando la puntuación de validación no esté mejorando en todos los n_iter_no_change anteriores números de iteraciones.

* `tol`: float, default=1e-4 Tolerancia a la parada anticipada. Cuando la pérdida no mejora en al menos `tol` para `n_iter_no_change` iteraciones (si se establece en un número), el entrenamiento se detiene.

* `ccp_alpha`: non-negative float, default=0.0 Parámetro de complejidad utilizado para la poda Minimal Cost-Complexity. Se elegirá el subárbol con la mayor complejidad de costo siendo más pequeño que ccp_alpha. De forma predeterminada, no se realiza ninguna poda.

Para `GradientBoosting` teniamos una grán cantidad de hiperparametros para seleccionar, aunque nosotros unicamente vamos a utilizar `criterion`, `learning_rate`, `max_depth`, `n_estimators` y `subsample`.

### HistGradientBoostingClassifier

Este estimador tiene soporte nativo para valores perdidos (NaN). Durante el entrenamiento, cuando el arbol crece, aprende en cada punto de división si las muestras con valores perdidos deben ir al hijo izquierdo o derecho, según la ganancia potencial. Al predecir, las muestras con valores perdidos se asignan al hijo izquierdo o derecho en consecuencia. Si no se encontraron valores perdidos para una característica determinada durante el entrenamiento, las muestras con valores perdidos se asignan al hijo que tenga más muestras.

Como el algoritmo Histogram Gradient Boosting realiza un discretizado interno, no permite el uso de discretizadores en pasos previos del pipeline, por lo que no utilizamos nuestro discretizador.

In [ ]:
estimator = HistGradientBoostingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_wisconsin',del_columns_wisconsin),('nan_anomalos_wisconsin', nan_anomalos_wisconsin),
                           ('imp', imp),('HistGradientBoosting', estimator)])

learning_rate = [0.01, 0.05, 0.1]
#max_leaf_nodes = [15, 31, 65, 127]
#max_depth = [2, 3, 4, 5, 6]
min_samples_leaf = [10, 20, 30]
loss = ["binary_crossentropy", "auto"]
max_iter = [50, 100, 150]
max_bins = [200, 255, 300, 350]

hist_gradient_boosting_clf = utils.optimize_params(pipeline,X_Cancer, y_Cancer, cv,
                                                   scoring=scoring,
                                                   HistGradientBoosting__learning_rate=learning_rate,
                                                   HistGradientBoosting__min_samples_leaf=min_samples_leaf,
                                                   HistGradientBoosting__loss=loss,
                                                   HistGradientBoosting__max_iter=max_iter)

* `loss`: {‘auto’, ‘binary_crossentropy’, ‘categorical_crossentropy’}, default=’auto’ La función de pérdida para usar en el proceso de refuerzo. "Binary_crossentropy" (también conocido como pérdida logística) se utiliza para la clasificación binaria y se generaliza a "categorical_crossentropy" para la clasificación multiclase. "Auto" elegirá automáticamente cualquiera de las pérdidas dependiendo de la naturaleza del problema.

* `learning_rate`: float, default=0.1 La tasa de aprendizaje, también conocida como contracción. Esto se usa como factor multiplicativo para los valores de las hojas. Utilice 1 para que no se encoja.

* `max_iter`: int, default=100 El número máximo de iteraciones del proceso de refuerzo, es decir, el número máximo de árboles para la clasificación binaria. Para la clasificación multiclase, se construyen n árboles de clases por iteración.

* `max_leaf_nodes`: int or None, default=31 El número máximo de hojas de cada árbol. Debe ser mayor que 1. Si es None, no hay límite máximo.

* `max_depth`: int or None, default=None La profundidad máxima de cada árbol.

* `min_samples_leaf`: int, default=20 El número mínimo de muestras por hoja. Para conjuntos de datos pequeños con menos de unos pocos cientos de muestras, se recomienda reducir este valor, ya que solo se construirían árboles muy poco profundos.

* `l2_regularization`: float, default=0 El parámetro de regularización L2. Utiliza 0 para no regularización.

* `max_bins`: int, default=255 El número máximo de contenedores que se utilizarán para los valores que no faltan. Antes del entrenamiento, cada característica de la matriz de entrada X se agrupa en bins con valores enteros, lo que permite una etapa de entrenamiento mucho más rápida. Las funciones con una pequeña cantidad de valores únicos pueden usar menos de max_bins bins. Además de los contenedores max_bins, siempre se reserva un contenedor más para los valores perdidos. No debe ser mayor de 255.

* `categorical_features`: array-like of {bool, int} of shape (n_features) or shape (n_categorical_features,), default=None. Indica las características categóricas.

  Ninguno: ninguna característica se considerará categórica.

  tipo matriz booleana: máscara booleana que indica características categóricas.

  tipo matriz de enteros: índices enteros que indican características categóricas.

    Para cada característica categórica, debe haber como máximo max_bins categorías únicas, y cada valor categórico debe estar en [0, max_bins -1].

* `monotonic_cst`: array-like of int of shape (n_features), default=None Indica la restricción monótona que se debe aplicar a cada función. -1, 1 y 0 corresponden respectivamente a una restricción negativa, una restricción positiva y ninguna restricción.

* `warm_start`: bool, default=False Cuando se establece en Verdadero, reutiliza la solución de la llamada anterior para ajustar y agregar más estimadores al conjunto. Para que los resultados sean válidos, el estimador debe volver a entrenarse solo con los mismos datos.

* `early_stopping`: ‘auto’ or bool, default=’auto’ Si es "auto", la parada anticipada está habilitada si el tamaño de la muestra es mayor que 10000. Si es Verdadero, la parada anticipada está habilitada; de lo contrario, la parada anticipada está deshabilitada.

* `scoring`: str or callable or None, default=’loss’ Parámetro de puntuación que se utilizará para la parada anticipada. Puede ser una sola cadena o un invocable. Si es None, se usa el puntaje predeterminado del estimador. Si la puntuación = 'loss', se verifica la detención anticipada con el valor de la pérdida. Solo se utiliza si se realiza una parada anticipada.

* `validation_fraction`: int or float or None, default=0.1 Proporción (o tamaño absoluto) de los datos de entrenamiento que se deben reservar como datos de validación para la detención anticipada. Si es None, la detención anticipada se realiza en los datos de entrenamiento. Solo se utiliza si se realiza una parada anticipada.

* `n_iter_no_change`: int, default=10 Utilizado para saber cuando hacer una parada anticipada. El proceso de ajuste se detiene cuando ninguna de las últimas puntuaciones n_iter_no_change es mejor que la n_iter_no_change - 1 -th-to-last one, hasta cierta tolerancia. Solo se utiliza si se realiza una parada anticipada.

* `tol`: float or None, default=1e-7 La tolerancia absoluta a utilizar al comparar puntuaciones. Cuanto mayor sea la tolerancia, más probabilidades hay de que nos detengamos antes de tiempo: una mayor tolerancia significa que será más difícil que las iteraciones posteriores se consideren una mejora en la puntuación de referencia.

* `verbose`: int, default=0 El nivel de verbosidad. Si no es cero, imprima alguna información sobre el proceso de ajuste.

* `random_state`: int, RandomState instance or None, default=None Generador de números pseudoaleatorios para controlar el submuestreo en el proceso de agrupamiento y la división de datos de validación/entrenamiento si la parada anticipada está habilitada. Pase un int para una salida reproducible a través de múltiples llamadas a funciones.

Respecto a `HistGradientBoosting` tambien tenemos una gran cantidad de hiperparametros que podriamos utilizar aunque nosotros unicamente hemos utilizado `learning_rate`, `loss`, `max_iter` ,`min_samples_leaf` y `max_bins`

## Selección para Pima

In [ ]:
scoring = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label=1)}

Tras haber explicado detalladamente los hiperparametros de los distintos algoritmos anteriormente, en la seleccion para Wisconsin, aqui no vamos a explicar mucho, luego comentaremos los resultados en la parte final.

### KNeighborsClassifier

In [ ]:
estimator = KNeighborsClassifier()
pipeline = Pipeline(steps=[('del_columns_pima',del_columns_pima),('nan_anomalos_pima', nan_anomalos_pima),('imp', imp),
                           ('discretizer_pima', discretizer_pima),('knn', estimator)])


X_Diabetes_Norm = normalize(X_Diabetes)


weights = ["uniform", "distance"]
n_neighbors = [3,5,7,9,11,13,15,17,19]
#algorithm = ["auto", "ball_tree", "kd_tree", "brute"]
#p =[2, 3, 4, 5, 6, 7]
#leaf_size = [30, 40, 50, 60, 70, 80, 90, 100]
metric = ["euclidean", "manhattan"]
#n_jobs = default -1

k_neighbors_clf_pima = utils.optimize_params(pipeline,X_Diabetes_Norm, y_Diabetes, cv,
                                             scoring=scoring,
                                             knn__weights=weights,
                                             knn__n_neighbors=n_neighbors)

### DecisionTreeClassifier

In [ ]:
estimator = DecisionTreeClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_pima',del_columns_pima),('nan_anomalos_pima', nan_anomalos_pima),('imp', imp),
                           ('discretizer_pima', discretizer_pima),('DecisionTree', estimator)])

criterion = ["gini", "entropy"]
max_depth = [4, 5, 6, 7, 8, 9, 10, None]
ccp_alpha = [0.0, 0.1, 0.2, 0.3]

#class_weight = ["balanced", None]
#max_features = [int, float, "auto", "sqrt", "log2", None]
#max_leaf_nodes = [8, 9, 10, 11, 12]
#min_impurity_decrease = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
#min_samples_split = [2, 3, 4, 5, 6, 7, 8, 9, 10]
#min_samples_leaf = [5,6,7,8,9,10,11,12,13,14,15]
#min_weight_fraction_leaf = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
#splitter = ["best","random"]





decision_tree_clf_pima = utils.optimize_params(pipeline,X_Diabetes, y_Diabetes, cv,
                                               scoring=scoring,
                                               DecisionTree__criterion=criterion,
                                               DecisionTree__max_depth=max_depth,
                                               DecisionTree__ccp_alpha=ccp_alpha)

### AdaBoostClassifier

In [ ]:
estimator = AdaBoostClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_pima',del_columns_pima),('nan_anomalos_pima', nan_anomalos_pima),('imp', imp),
                           ('discretizer_pima', discretizer_pima),('AdaBoost', estimator)])

# Should not modify the base original model
base_estimator = DecisionTreeClassifier(random_state=random_state)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]
#n_estimators = [100, 150, 200]

criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]


adaboost_clf_pima = utils.optimize_params(pipeline,X_Diabetes, y_Diabetes, cv,
                                          scoring=scoring,
                                          AdaBoost__base_estimator=base_estimator,
                                          AdaBoost__learning_rate=learning_rate,
                                          AdaBoost__base_estimator__criterion=criterion,
                                          AdaBoost__base_estimator__max_depth=max_depth,
                                          AdaBoost__base_estimator__ccp_alpha=ccp_alpha)

### BaggingClassifier

In [ ]:
estimator = BaggingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_pima',del_columns_pima),('nan_anomalos_pima', nan_anomalos_pima),('imp', imp),
                           ('discretizer_pima', discretizer_pima),('Bagging', estimator)])

# Should not modify the base original model
base_estimator = DecisionTreeClassifier(random_state=random_state)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
max_depth = [4, 5, 6, 7, 8]
min_samples_leaf = [5,6,7,8,9,10,11,12,13,14,15]

bagging_clf_pima = utils.optimize_params(pipeline,X_Diabetes, y_Diabetes, cv,
                                         scoring=scoring,
                                         Bagging__base_estimator=base_estimator,
                                         Bagging__base_estimator__criterion=criterion,
                                         Bagging__base_estimator__max_depth=max_depth,
                                         Bagging__base_estimator__min_samples_leaf=min_samples_leaf)

### RanfomForestClassifier

In [ ]:
estimator = RandomForestClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_pima',del_columns_pima),('nan_anomalos_pima', nan_anomalos_pima),('imp', imp),
                           ('discretizer_pima', discretizer_pima),('RandForest', estimator)])

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
max_depth = [5,6,7,8,9]

random_forest_clf_pima = utils.optimize_params(pipeline,X_Diabetes, y_Diabetes, cv,
                                               scoring=scoring,
                                               RandForest__criterion=criterion,
                                               RandForest__max_features=max_features,
                                               RandForest__max_depth=max_depth)

### GradientBoostingClassifier

In [ ]:
estimator = GradientBoostingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_pima',del_columns_pima),('nan_anomalos_pima', nan_anomalos_pima),('imp', imp),
                           ('discretizer_pima', discretizer_pima),('GradBoosting', estimator)])

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
#ccp_alpha = [0.0, 0.1]
n_estimators = [50, 100, 200]
subsample = [0.2, 0.8, 1.0]

gradient_boosting_clf_pima = utils.optimize_params(pipeline,X_Diabetes, y_Diabetes, cv,
                                                   scoring=scoring,
                                                   GradBoosting__learning_rate=learning_rate,
                                                   GradBoosting__criterion=criterion,
                                                   GradBoosting__max_depth=max_depth,
                                                   GradBoosting__subsample=subsample,
                                                   GradBoosting__n_estimators = n_estimators)

### HistGradientBoostingClassifier

In [ ]:
estimator = HistGradientBoostingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns_pima',del_columns_pima),('nan_anomalos_pima', nan_anomalos_pima),('imp', imp),('HistGradBoosting', estimator)])

learning_rate = [0.01, 0.05, 0.1]
#max_leaf_nodes = [15, 31, 65, 127]
#max_depth = [2, 3, 4, 5, 6]
min_samples_leaf = [10, 20, 30]
loss = ["binary_crossentropy", "auto"]
max_iter = [50, 100, 150]
max_bins = [200, 255, 300, 350]

hist_gradient_boosting_clf_pima = utils.optimize_params(pipeline,X_Diabetes, y_Diabetes, cv,
                                                        scoring=scoring,
                                                        HistGradBoosting__learning_rate=learning_rate,
                                                        HistGradBoosting__min_samples_leaf=min_samples_leaf,
                                                        HistGradBoosting__loss=loss,
                                                        HistGradBoosting__max_iter=max_iter)

# 5. Construcción y validación del modelo final

Ya tenemos todos los clasificadores optimizados, por lo que ahora tendremos que elegir el mejor de todos. 
Como es complicado ver todos los resultados en el apartado anterior, gracias a las funciones de evaluate_estimators2 vamos a poder ver de manera clara los resultados obtenidos por los distintos clasificadores en unas tablas.

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf 
}

In [ ]:
X_Cancer = X_test_Cancer
y_Cancer = y_test_Cancer

utils.evaluate_estimators2(estimators, X_Cancer, y_Cancer, 'M')

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf_pima,
    "Decision tree": decision_tree_clf_pima,
    "AdaBoost": adaboost_clf_pima,
    "Bagging": bagging_clf_pima,
    "Random Forests": random_forest_clf_pima,
    "Gradient Boosting": gradient_boosting_clf_pima,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf_pima
}

In [ ]:
X_Diabetes = X_test_Diabetes
y_Diabetes = y_test_Diabetes

utils.evaluate_estimators2(estimators, X_Diabetes, y_Diabetes,1)

Viendo estos datos, el modelo que peor resultados brinda es el de `Nearest neighbors`. Pensamos que esto es así porque tenemos bastantes variables y KNN se comporta de forma que todas las variables se consideran de igual importancia, lo que hace que una variable que aporte poco al problema le quite importancia a una variable que sea significativa.

Podemos observar que `Histogram Gradient Boosting` obtiene los mejores resultados en Wisconsin pero en Pima quien obtiene los mejores resultados es `Decision Tree Classifier`, esto puede deberse por la gran diferencia que hay entre las dos bases de datos en cuanto a tamaño y cantidad de información.

En cuanto a los ensembles vemos que generalmente obtienen buenos resultados en general.





# 6. Titanic

Cargamos los datos y obtenemos los conjuntos de variables predictoras X y la variable clase y.

In [ ]:
filepath = "../input/titanic/train.csv"

index = "PassengerId"
target = "Survived"

Titanic_data = utils.load_data(filepath, index, target)

In [ ]:
Titanic_data.sample(5, random_state=random_state)

In [ ]:
Titanic_data["Survived"]=Titanic_data["Survived"].astype("category")
Titanic_data.info(memory_usage=False)

In [ ]:
(X_Titanic_data, y_Titanic_data)=utils.divide_dataset(Titanic_data,target="Survived")

In [ ]:
X_Titanic_data.sample(5, random_state=random_state)

In [ ]:
y_Titanic_data.sample(5, random_state=random_state)

Vemos que la particion de la base de datos en variables predictoras y por otro lado la variable clase se ha realizado correctamente, por lo que ahora dividimos en train/test.

In [ ]:
(X_Titanic_data_train,X_Titanic_data_test,y_Titanic_data_train,y_Titanic_data_test)=train_test_split(X_Titanic_data,
                                                                                                     y_Titanic_data,
                                                                                                     stratify=y_Titanic_data,
                                                                                                     random_state=random_state,
                                                                                                     train_size=0.7)

In [ ]:
X_Titanic = X_Titanic_data_train
y_Titanic = y_Titanic_data_train

In [ ]:
X_Titanic.sample(5, random_state=random_state)

Creamos el Pipeline para titanic obtenido de la practica anterior:

In [ ]:
del_columns = utils.QuitarColumnasTransformer(['Cabin', 'Name', 'Ticket', 'Fare'])

string_int1 = utils.StringAIntTransformer('Sex',['female','male'],[0,1])
string_int2 = utils.StringAIntTransformer('Embarked',['Q','C','S'],[1,2,3])

nan_anomalos_titanic = utils.AnomalosANanTransformer({'Pclass':[1,3], 'Sex':[0,1], 'SibSp':[0, 8],
                                         'Parch':[0, 6], 'Embarked':[1, 3] })

imp = SimpleImputer(missing_values=float('nan'), strategy='most_frequent')

discretizer_titanic = KBinsDiscretizer(n_bins=2, strategy="uniform")

Creamos la funcion para normalizar la base de datos de titanic.

In [ ]:
def normalizeTitanic(dataset):
    dataNorm=((dataset["Pclass"])/(dataset["Pclass"].max()))
    dataNorm=((dataset["SibSp"])/(dataset["SibSp"].max()))
    dataNorm=((dataset["Parch"])/(dataset["Parch"].max()))
    
    dataNorm["Sex"]=dataset["Sex"]
    dataNorm["Embarked"]=dataset["Embarked"]
    
    dataNorm["Age"]=dataset["Age"]
    dataNorm["Cabin"]=dataset["Cabin"]
    dataNorm["Name"]=dataset["Name"]
    dataNorm["Ticket"]=dataset["Ticket"]
    dataNorm["Fare"]=dataset["Fare"]
    return dataNorm

Elegimos los scorers que vamos a utilizar.

In [ ]:
scoring = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label=1)}

Empezamos a optimizar los hiperparametros de los modelos y comentaremos los resultados al final de todo.

In [ ]:
estimator = KNeighborsClassifier()
pipeline = Pipeline(steps=[('del_columns',del_columns),('string_int1', string_int1),('string_int2',string_int2),
                           ('nan_anomalos_titanic',nan_anomalos_titanic),('imp', imp),
                           ('discretizer_titanic', discretizer_titanic),('knn', estimator)])


X_Titanic_Norm = normalizeTitanic(X_Titanic)


weights = ["uniform", "distance"]
n_neighbors = [3,5,7,9,11,13,15,17,19]
#algorithm = ["auto", "ball_tree", "kd_tree", "brute"]
#p =[2, 3, 4, 5, 6, 7]
#leaf_size = [30, 40, 50, 60, 70, 80, 90, 100]
metric = ["euclidean", "manhattan"]
#n_jobs = default -1

k_neighbors_clf_titanic = utils.optimize_params(pipeline,X_Titanic, y_Titanic, cv,
                                             scoring=scoring,
                                             knn__weights=weights,
                                             knn__n_neighbors=n_neighbors,
                                             knn__metric=metric)

In [ ]:
estimator = DecisionTreeClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns',del_columns),('string_int1', string_int1),('string_int2',string_int1),
                           ('nan_anomalos_titanic',nan_anomalos_titanic),('imp', imp),
                           ('discretizer_titanic', discretizer_titanic),('DecisionTree', estimator)])



criterion = ["gini", "entropy"]
max_depth = [4, 5, 6, 7, 8, 9, 10, None]
ccp_alpha = [0.0, 0.1, 0.2, 0.3]

#class_weight = ["balanced", None]
#max_features = [int, float, "auto", "sqrt", "log2", None]
#max_leaf_nodes = [8, 9, 10, 11, 12]
#min_impurity_decrease = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
#min_samples_split = [2, 3, 4, 5, 6, 7, 8, 9, 10]
#min_samples_leaf = [5,6,7,8,9,10,11,12,13,14,15]
#min_weight_fraction_leaf = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
#splitter = ["best","random"]

decision_tree_clf_titanic = utils.optimize_params(pipeline,X_Titanic, y_Titanic, cv,
                                             scoring=scoring,
                                             DecisionTree__criterion=criterion,
                                             DecisionTree__max_depth=max_depth,
                                             DecisionTree__ccp_alpha=ccp_alpha)

In [ ]:
estimator = AdaBoostClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns',del_columns),('string_int1', string_int1),('string_int2',string_int1),
                           ('nan_anomalos_titanic',nan_anomalos_titanic),('imp', imp),
                           ('discretizer_titanic', discretizer_titanic),('adaBoost', estimator)])


# Should not modify the base original model
base_estimator = DecisionTreeClassifier(random_state=random_state)

base_estimator = [base_estimator]
learning_rate = [0.95, 1.0]
#n_estimators = [100, 150, 200]

criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

adaboost_clf_titanic = utils.optimize_params(pipeline,X_Titanic, y_Titanic, cv,
                                             scoring=scoring,
                                             adaBoost__learning_rate=learning_rate,
                                             adaBoost__base_estimator=base_estimator,
                                             adaBoost__base_estimator__criterion=criterion,
                                             adaBoost__base_estimator__max_depth=max_depth,
                                             adaBoost__base_estimator__ccp_alpha=ccp_alpha)

In [ ]:
estimator = BaggingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns',del_columns),('string_int1', string_int1),('string_int2',string_int1),
                           ('nan_anomalos_titanic',nan_anomalos_titanic),('imp', imp),
                           ('discretizer_titanic', discretizer_titanic),('bagging', estimator)])


base_estimator = DecisionTreeClassifier(random_state=random_state)

base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
max_depth = [4, 5, 6, 7, 8]
min_samples_leaf = [5,6,7,8,9,10,11,12,13,14,15]


bagging_clf_titanic = utils.optimize_params(pipeline,X_Titanic, y_Titanic, cv,
                                             scoring=scoring,
                                             bagging__base_estimator=base_estimator,
                                             bagging__base_estimator__criterion=criterion,
                                             bagging__base_estimator__max_depth=max_depth,
                                             bagging__base_estimator__min_samples_leaf=min_samples_leaf)

In [ ]:
estimator = RandomForestClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns',del_columns),('string_int1', string_int1),('string_int2',string_int1),
                           ('nan_anomalos_titanic',nan_anomalos_titanic),('imp', imp),
                           ('discretizer_titanic', discretizer_titanic),('RFC', estimator)])

criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
#max_depth = [5,6,7,8,9]

random_forest_clf_titanic = utils.optimize_params(pipeline,X_Titanic, y_Titanic, cv,
                                             scoring=scoring,
                                             RFC__criterion=criterion,
                                             RFC__max_features=max_features)

In [ ]:
estimator = GradientBoostingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns',del_columns),('string_int1', string_int1),('string_int2',string_int1),
                           ('nan_anomalos_titanic',nan_anomalos_titanic),('imp', imp),
                           ('discretizer_titanic', discretizer_titanic),('gradBoosting', estimator)])

learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
#ccp_alpha = [0.0, 0.1]
n_estimators = [50, 100, 200]
subsample = [0.2, 0.8, 1.0]

gradient_boosting_clf_titanic = utils.optimize_params(pipeline,X_Titanic, y_Titanic, cv,
                                             scoring=scoring,
                                             gradBoosting__learning_rate=learning_rate,
                                             gradBoosting__criterion=criterion,
                                             gradBoosting__max_depth=max_depth,
                                             gradBoosting__n_estimators=n_estimators,
                                             gradBoosting__subsample=subsample)

In [ ]:
estimator = HistGradientBoostingClassifier(random_state=random_state)
pipeline = Pipeline(steps=[('del_columns',del_columns),('string_int1', string_int1),('string_int2',string_int1),
                           ('nan_anomalos_titanic',nan_anomalos_titanic),('imp', imp),('histGradBoosting', estimator)])





learning_rate = [0.01, 0.05, 0.1]
#max_leaf_nodes = [15, 31, 65, 127]
#max_depth = [2, 3, 4, 5, 6]
min_samples_leaf = [10, 20, 30]
loss = ["binary_crossentropy", "auto"]
max_iter = [50, 100, 150]

hist_gradient_boosting_clf_titanic = utils.optimize_params(pipeline,X_Titanic, y_Titanic, cv,
                                             scoring=scoring,
                                             histGradBoosting__learning_rate=learning_rate,
                                             histGradBoosting__min_samples_leaf=min_samples_leaf,
                                             histGradBoosting__loss=loss,
                                             histGradBoosting__max_iter=max_iter)

Finalmente construimos y validamos el modelo final.

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf_titanic,
    "Decision tree": decision_tree_clf_titanic,
    "AdaBoost": adaboost_clf_titanic,
    "Bagging": bagging_clf_titanic,
    "Random Forests": random_forest_clf_titanic,
    "Gradient Boosting": gradient_boosting_clf_titanic,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf_titanic 
}

In [ ]:
X_Titanic = X_Titanic_data_test
y_Titanic = y_Titanic_data_test

utils.evaluate_estimators2(estimators, X_Titanic, y_Titanic,1)

Con los hiperparametros optimizados podemos observar que el mejor modelo es el `Histogram Gradient Boosting` en cuanto a ROC_AUC_Score, en cuanto a Recall_score el mejor modelo es el de `KNN` 

# Enlace al kernel donde hemos realizado el estudio:
 
 https://www.kaggle.com/pablomoreiragarcia/estudio-kernel